# An Assortment of Julia Interfaces for Accelerators

## GPUs, Clusters, Multithreading, and fancy tools for old boring CPUs

- Tools for interfacing with hardware accelerators
- Tools for Cluster multi-processing, or single node multi-threading
- Tools to make the best use of your CPU

In [21]:
using BenchmarkTools # You should always use this library
using LinearAlgebra

# Julia GPU interfaces

`CUDA` for Nvidia hardware is well supported. `AMDGPU` for AMD hardware is not reliable yet.

See also: [The Julia GPU documentation](https://juliagpu.org/)

In [23]:
using AMDGPU # For AMD GPUs, experimental, not always worth it
toGPU = ROCArray;

In [ ]:
using CUDA # For Nvidia GPUs, fairly mature 
toGPU = CuArray;

In [ ]:
using oneAPI # An Intel attempt at unified driver landscape, not used by many
toGPU = oneArray;

There is also some stuff about TPUs and other special hardware.

In [ ]:
N = 5000
t = Float32
A = rand(t,N,N)
B = rand(t,N,N)
C = rand(t,N,N)
Ag = toGPU(A)
Bg = toGPU(B)
Cg = toGPU(C);

In [ ]:
@btime mul!(C,A,B)
@btime mul!(Cg,Ag,Bg);

In [26]:
n = 100_000_000
t = Float32
a = rand(t,n)
b = rand(t,n)
c = rand(t,n)
ag = toGPU(a)
bg = toGPU(b)
cg = toGPU(c);

In [27]:
@btime c .= sin.(b .* a)
@btime cg .= sin.(bg .* ag);

  468.857 ms (4 allocations: 128 bytes)
  11.442 ms (126 allocations: 6.02 KiB)


## GPU ODE example

See also: [DifferentialEquations.jl FAQ on GPU usage](https://diffeq.sciml.ai/stable/basics/faq/#GPUs,-multithreading-and-distributed-computation-support)

In [ ]:
using OrdinaryDiffEq

n = 1000
t = Float32
u0 = toGPU(rand(t,n))
A  = toGPU(randn(t,n,n))
f(du,u,p,t)  = mul!(du,A,u)

tspan = (0.0,1.0)
tspan = t.(tspan)

prob = ODEProblem(f,u0,tspan)
solver_algo = Tsit5() # You might want to be careful with the choice of algo
sol = solve(prob,)

# Julia Distributed Computing interface

Julia includes tools to run computation on clusters of computers

- [Julia manual on distributed capabilities](https://docs.julialang.org/en/v1/manual/distributed-computing/)
- [The Base distrubuted computing library `Distrubuted.jl`](https://docs.julialang.org/en/v1/stdlib/Distributed/)
- [`DistributedArrays.jl` for gigantic arrays split between devices](https://juliaparallel.github.io/DistributedArrays.jl/stable/)
- [A toy example you can run on a single computer](https://juliaparallel.github.io/DistributedArrays.jl/stable/#Example-1)

In [28]:
using Distributed

# Adding 3 processes to the cluster
addprocs(3)

# Running setup code on each process
@everywhere using LinearAlgebra

In [29]:
# Requesting an arbitrary worker to perform a computation
# This is async / non-blocking command
references_to_computations = @spawnat :any norm(rand(1000))/1000

Future(2, 1, 11, nothing)

In [30]:
# Getting the result
computation_results = fetch(references_to_computations)

0.01793967931340792

In [50]:
fetch(@spawnat :any myid())

4

In [51]:
work_function(n) = norm(rand(n))/n
pmap(work_function, [100,10000,900,30])

LoadError: On worker 2:
UndefVarError: #work_function not defined
Stacktrace:
  [1] [0m[1mdeserialize_datatype[22m
[90m    @ [39m[90m/buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Serialization/src/[39m[90;4mSerialization.jl:1280[0m
  [2] [0m[1mhandle_deserialize[22m
[90m    @ [39m[90m/buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Serialization/src/[39m[90;4mSerialization.jl:827[0m
  [3] [0m[1mdeserialize[22m
[90m    @ [39m[90m/buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Serialization/src/[39m[90;4mSerialization.jl:774[0m
  [4] [0m[1mhandle_deserialize[22m
[90m    @ [39m[90m/buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Serialization/src/[39m[90;4mSerialization.jl:834[0m
  [5] [0m[1mdeserialize[22m
[90m    @ [39m[90m/buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Serialization/src/[39m[90;4mSerialization.jl:774[0m[90m [inlined

In [52]:
@everywhere work_function(n) = norm(rand(n))/n
pmap(work_function, [100,10000,900,30])

4-element Vector{Float64}:
 0.055641432124401556
 0.005791920133203985
 0.01904387958085956
 0.09539130437047112

[`DistributedArrays.jl`](https://juliaparallel.github.io/DistributedArrays.jl/) can be used to create arrays that are distributed among many cluster nodes. It works seamlessly and virtually all Julia functions work on these arrays as if they were local arrays (or GPU arrays). However, for it to work fast, you need to put a lot of thought into exactly how the array is going to be partitioned.

# Julia Multithreading interface

You need to start Julia with the command `julia --threads=N` for N threads or `--threads=auto` to automatically use all CPU cores.

See also:

- [Official Blog post on the topic](https://julialang.org/blog/2019/07/multithreading/)
- [Manual](https://docs.julialang.org/en/v1/manual/multi-threading/)
- [Multithreading module documentation](https://docs.julialang.org/en/v1/base/multi-threading/)

In [53]:
using Base.Threads

# Check how many threads are available;
# These examples will not be interesting on a single thread
Threads.nthreads()

1

The `@threads` macro lets you automatically multithread a loop.

In [ ]:
a = zeros(10)
@threads for i in 1:10
   a[i] = threadid()
end
a

In [ ]:
some_interesting_computation(p) = sin(p)^2

@threads for parameter in [0.5,0.6,0.7]
   println(some_interesting_computation(parameter))
end

The `ThreadTools` library has some extra convenience functions. Unlike other libraries discussed here, it is rather small and it might be superseeded by functionality included in base Julia.

In [ ]:
using ThreadTools # A small independent package / some of this might end up in base Julia

tmap(some_interesting_computation, [0.5, 0.6, 0.7])

Threading is simple and great if the loop iterations are independent, but it might cause immense suffering if you are not careful with data dependencies.

FYI, assigning to a particular index in an array works fine in threads, so just do not use `push!`.

In [ ]:
function test()
    a = []
    @threads for i in 1:100000
        push!(a,1)
    end
    length(a)
end
test()

# Fast Loops and Tensor Operations (on CPUs and maybe more)

This will get pretty low level...

## Low haging fruits

- [Julia Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)
- [`@inbounds`, `@fastmath`, `@simd`](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-annotations)
- [`StaticArray`](https://github.com/JuliaArrays/StaticArrays.jl) if you have many small arrays

In [67]:
x = rand(10000)
y = rand(10000)

function vecprod1(x,y)
    s = 0.0
    for i in eachindex(x)
        s += x[i]*y[i]
    end
    s
end

@btime vecprod1(x,y);

  9.447 μs (1 allocation: 16 bytes)


In [68]:
function vecprod2(x,y)
    s = 0.0
    @inbounds @simd for i in eachindex(x)
        s += x[i]*y[i]
    end
    s
end

@btime vecprod2(x,y);

  1.822 μs (1 allocation: 16 bytes)


## Fancy Tensor Libraries

Mostly based around [LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl).

| Julia Package                                                    | CPU | GPU | Note                        |
| ---------------------------------------------------------------- | --- | --- |:--------------------------- |
| [GemmKernels.jl](https://github.com/JuliaGPU/GemmKernels.jl)     | No  | Yes | the fastest CPU performance |
| [Octavian.jl](https://github.com/JuliaLinearAlgebra/Octavian.jl) | Yes | No  | the fastest CPU performance |
| [Tullio.jl](https://github.com/mcabbott/Tullio.jl)               | Yes | Yes | the most flexible           |


In [ ]:
using Tullio

@tullion A[i,j] = B[i,k]*C[k,j]

@tullio M[x,y,c] := N[x+i, y+j,c] * K[i,j]     # sum over i,j, and create M

@tullio S[x] = P[x,y] * log(Q[x,y] / R[y])     # sum over y, and write into S

@tullio A[i,j] += B[i,k,l] * C[l,j] * D[k,j]   # sum over k,l, and add to values in A

@tullio (*) Z[j] := X[ind[k],j] * exp(-Y[k])   # product over k